In [1]:
import pandas
import konlpy
import gensim

print(pandas.__version__)
print(konlpy.__version__)
print(gensim.__version__)

1.3.3
0.5.2
4.1.2


In [2]:
import torch
import pandas as pd
from PIL import Image
import glob
import matplotlib.pyplot as plt
import os
import numpy as np

import random
import torch.backends.cudnn as cudnn

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
np.random.seed(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

In [3]:
data = pd.read_csv('~/aiffel/project/FnGuide/finance_data.csv')

In [4]:
data = data[['labels', 'kor_sentence']]
data

,labels,kor_sentence
0,neutral,"Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로..."
1,neutral,테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평...
2,negative,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ..."
3,positive,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...
4,positive,"2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 4..."
...,...,...
4841,negative,런던 마켓워치 -- 은행주의 반등이 FTSE 100지수의 약세를 상쇄하지 못하면서 ...
4842,neutral,린쿠스키아의 맥주 판매량은 416만 리터로 6.5% 감소했으며 카우노 알루스의 맥주...
4843,negative,"영업이익은 2007년 68.8 mn에서 35.4 mn으로 떨어졌으며, 선박 판매 이..."
4844,negative,페이퍼 부문 순매출은 2008년 2분기 241.1 mn에서 2009년 2분기 221...


In [12]:
data['labels'] = data['labels'].replace({'negative': 0, 'neutral' : 1, 'positive': 2})
data

/tmp/ipykernel_304/3547878446.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['labels'] = data['labels'].replace({'negative': 0, 'neutral' : 1, 'positive': 2})


,labels,kor_sentence
0,1,"Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로..."
1,1,테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평...
2,0,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ..."
3,2,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...
4,2,"2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 4..."
...,...,...
4841,0,런던 마켓워치 -- 은행주의 반등이 FTSE 100지수의 약세를 상쇄하지 못하면서 ...
4842,1,린쿠스키아의 맥주 판매량은 416만 리터로 6.5% 감소했으며 카우노 알루스의 맥주...
4843,0,"영업이익은 2007년 68.8 mn에서 35.4 mn으로 떨어졌으며, 선박 판매 이..."
4844,0,페이퍼 부문 순매출은 2008년 2분기 241.1 mn에서 2009년 2분기 221...


In [13]:
data = data[~data['kor_sentence'].duplicated()]

In [14]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
import pandas as pd

In [15]:
sentences = data['kor_sentence'].tolist()
labels = data['labels'].tolist()

# Train-Test 데이터 분할
train_sentences, test_sentences, train_labels, test_labels = train_test_split(sentences, labels, test_size=0.2, random_state=123)

# Tokenizer 생성
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base")

# 문장을 토큰으로 변환
train_inputs = tokenizer.batch_encode_plus(
    train_sentences,
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt"
)
test_inputs = tokenizer.batch_encode_plus(
    test_sentences,
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="pt"
)

# 레이블을 텐서로 변환
train_labels = torch.tensor(train_labels)
test_labels = torch.tensor(test_labels)

# 데이터를 DataLoader로 만듦
train_dataset = TensorDataset(train_inputs['input_ids'], train_inputs['attention_mask'], train_labels)
test_dataset = TensorDataset(test_inputs['input_ids'], test_inputs['attention_mask'], test_labels)

batch_size = 8
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 모델 생성 및 불러오기
model = RobertaForSequenceClassification.from_pretrained("klue/roberta-base", num_labels=3)

# GPU 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 옵티마이저 및 손실 함수 설정
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'clas

In [16]:
epochs = 10
for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    train_acc = 0.0
    
    for batch in train_dataloader:
        batch = tuple(t.to(device) for t in batch)
        inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}
        
        optimizer.zero_grad()
        outputs = model(**inputs)
        
        logits = outputs.logits
        loss = loss_fn(logits, inputs['labels'])
        
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = torch.max(logits, dim=1)
        train_acc += (predicted == inputs['labels']).sum().item()
        
    train_loss /= len(train_dataloader)
    train_acc /= len(train_dataset)
    
    print(f"Epoch {epoch+1}/{epochs}")
    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")

Epoch 1/10
Train Loss: 0.5700 | Train Acc: 0.7692
Test Loss: 0.4051 | Test Acc: 0.8292

Epoch 2/10
Train Loss: 0.3241 | Train Acc: 0.8778
Test Loss: 0.3687 | Test Acc: 0.8551

Epoch 3/10
Train Loss: 0.2189 | Train Acc: 0.9239
Test Loss: 0.4281 | Test Acc: 0.8458

Epoch 4/10
Train Loss: 0.1472 | Train Acc: 0.9485
Test Loss: 0.4520 | Test Acc: 0.8437

Epoch 5/10
Train Loss: 0.1017 | Train Acc: 0.9658
Test Loss: 0.6052 | Test Acc: 0.8333

Epoch 6/10
Train Loss: 0.0728 | Train Acc: 0.9769
Test Loss: 0.6595 | Test Acc: 0.8437

Epoch 7/10
Train Loss: 0.0451 | Train Acc: 0.9899
Test Loss: 0.5948 | Test Acc: 0.8416

Epoch 8/10
Train Loss: 0.0427 | Train Acc: 0.9881
Test Loss: 0.6085 | Test Acc: 0.8406

Epoch 9/10
Train Loss: 0.0500 | Train Acc: 0.9850
Test Loss: 0.6651 | Test Acc: 0.8478

Epoch 10/10
Train Loss: 0.0312 | Train Acc: 0.9907
Test Loss: 0.7896 | Test Acc: 0.8427



In [19]:
from sklearn.metrics import classification_report

model.eval()
test_loss = 0.0
test_acc = 0.0
y_true = []
y_pred = []

for batch in test_dataloader:
    batch = tuple(t.to(device) for t in batch)
    inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}
    
    with torch.no_grad():
        outputs = model(**inputs)
        
    logits = outputs.logits
    loss = loss_fn(logits, inputs['labels'])
    
    test_loss += loss.item()
    _, predicted = torch.max(logits, dim=1)
    test_acc += (predicted == inputs['labels']).sum().item()
    
    y_true.extend(inputs['labels'].tolist())
    y_pred.extend(predicted.tolist())

test_loss /= len(test_dataloader)
test_acc /= len(test_dataset)

report = classification_report(y_true, y_pred, target_names=['negative', 'neutral' ,'positive'])

print(f"Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.4f}")
print("Classification Report:\n", report)

Test Loss: 0.7896 | Test Accuracy: 0.8427
Classification Report:
               precision    recall  f1-score   support

    negative       0.83      0.72      0.77       128
     neutral       0.83      0.93      0.88       553
    positive       0.87      0.73      0.80       285

    accuracy                           0.84       966
   macro avg       0.85      0.79      0.82       966
weighted avg       0.84      0.84      0.84       966

